<a href="https://colab.research.google.com/github/rcastelhano/Machine-Learning-with-Python/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Add as the first cell
!pip uninstall -y tf-nightly keras-nightly protobuf

!pip install tensorflow==2.20.0 # Version doesn't matter much here, the default one will work as well - 2.19.0

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_csv(train_file_path, sep='\t')
df_test = pd.read_csv(test_file_path, sep='\t')
df_train.columns = ['label', 'message']
df_test.columns = ['label', 'message']
df_train_labels = pd.Categorical(df_train.label)
df_train_data = pd.Categorical(df_train.message)
df_test_labels = pd.Categorical(df_test.label)
df_test_data = pd.Categorical(df_test.message)
df_train['label'] = df_train_labels.codes
df_test['label'] = df_test_labels.codes

tokenizer = keras.preprocessing.text.Tokenizer(char_level=False, oov_token="<OOV>")
tokenizer.fit_on_texts(df_train['message'])
train_sequences = tokenizer.texts_to_sequences(df_train['message'])
maxlen = max(len(x) for x in train_sequences)
train_array = keras.preprocessing.sequence.pad_sequences(train_sequences, padding='post', maxlen=maxlen)
test_sequences = tokenizer.texts_to_sequences(df_test['message'])
test_array = keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=maxlen)
train_labels = df_train['label'].values
test_labels = df_test['label'].values

model = keras.Sequential()
model.add(keras.layers.Input(shape=(maxlen,)))
model.add(keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64)))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))


model.summary()


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_array, train_labels, epochs=10, validation_data=(test_array, test_labels), verbose=1)


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  pred_text = tokenizer.texts_to_sequences([pred_text])
  pred_array = keras.preprocessing.sequence.pad_sequences(pred_text, padding='post', maxlen=train_array.shape[1])
  prediction = model.predict(pred_array)

  prob = float(prediction[0][0])
  label = 'spam' if prob > 0.5 else 'ham'

  return [prob, label]

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
